In [14]:
import numpy as np
import pandas as pd

from typing import List
from queries import GET_CONEXION
import sqlite3

In [127]:
df_estaciones = pd.read_csv('./data/estaciones.csv', sep=";")
tabla = np.loadtxt('./data/tabla.csv', delimiter=",")
conn = sqlite3.connect("./data/estaciones.db")
cursor = conn.cursor()

In [129]:
def heuristica(node: int, destino: int) -> float:
    numeroNode = df_estaciones.loc[df_estaciones['ID'] == node].index[0]
    numeroDestino = df_estaciones.loc[df_estaciones['ID'] == destino].index[0]
    return tabla[numeroNode, numeroDestino]

In [259]:
def busqueda_camino(cursor, inicio, destino, hora=0) -> List[str]:
    # TODO Implementar el algoritmo de A* para que devuelva el camino más corto
    # NOTA: lo de **kwargs es para añadir más parámetro por si queremos ajustarlo a 
    # la hora del día u otra cosa.
    
    abierta = []
    cerrada = []
    
    if destino == inicio:
        return [(inicio, 0)]
    for d in cursor.execute("SELECT destino from conexiones where origen==?", [inicio]):
        if d[0] == destino:
            return [inicio, (destino, heuristica(inicio, d[0]))]
        else:
            gpeso = heuristica(inicio, d[0])
            hpeso = heuristica(d[0], destino)
            abierta += [(d[0], gpeso, hpeso, gpeso + hpeso, [inicio, d[0]])]
    
    encontrado = False
    while not encontrado:
        abierta.sort(key = lambda x: x[3]) 
        analizando = abierta.pop(0)
        cerrada.append(analizando[0])
        print([analizando[0], analizando[4][-2]])
        for adyacente in cursor.execute("SELECT destino from conexiones where origen==? AND destino != ?",
                                        [analizando[0], analizando[4][-2]]):
            lbool = list(map(lambda x: x[0] == adyacente[0], abierta))
            gpeso = analizando[1] + heuristica(analizando[0], adyacente[0])
            hpeso = heuristica(adyacente[0], destino)
            if adyacente[0] == destino:
                return analizando[4] + [(destino, analizando[1] + heuristica(analizando[0], destino))]
            elif adyacente[0] in cerrada:
                pass
            elif max(lbool):
                indice = lbool.index(True)
                if abierta[indice][1] < gpeso:
                    abierta[indice] = [(adyacente[0], gpeso, hpeso, gpeso+hpeso,
                                       analizando[4] + [adyacente[0]])]
            else:
                abierta += [(adyacente[0], gpeso, hpeso, gpeso+hpeso, analizando[4] + [adyacente[0]])]
    return cerrada

In [261]:
busqueda_camino(cursor, 113, 218)

[114, 113]
[115, 114]
[116, 115]
[117, 116]
[118, 117]
[119, 118]
[314, 119]
[315, 314]


[113, 114, 115, 116, 117, 118, 119, 314, 315, (218, 15.447690139709714)]

In [242]:
heuristica(119, 314)

0.18958026016602958

In [253]:
# metodo 2 por si hay que cambiar algo
def busqueda_camino2(cursor, inicio, destino, hora=0) -> List[str]:
    # TODO Implementar el algoritmo de A* para que devuelva el camino más corto
    # NOTA: lo de **kwargs es para añadir más parámetro por si queremos ajustarlo a 
    # la hora del día u otra cosa.
    
    abierta = []
    cerrada = []
    
    if destino == inicio:
        return [(inicio, 0)]
    for d in cursor.execute("SELECT destino from conexiones where origen==?", [inicio]):
        if d[0] == destino:
            return [(inicio, 0), (destino, heuristica(inicio, d[0]))]
        else:
            gpeso = heuristica(inicio, d[0])
            hpeso = heuristica(d[0], destino)
            abierta += [(d[0], gpeso, hpeso, gpeso + hpeso, [(inicio, 0), (d[0], heuristica(inicio, d[0]))])]
    
    encontrado = False
    while not encontrado:
        abierta.sort(key = lambda x: x[3]) 
        analizando = abierta.pop(0)
        cerrada.append(analizando[0])
        for adyacente in cursor.execute("SELECT destino from conexiones where origen==? AND destino != ?",
                                        [analizando[0], analizando[4][-2][0]]):
            lbool = list(map(lambda x: x[0] == adyacente[0], abierta))
            gpeso = analizando[1] + heuristica(analizando[0], adyacente[0])
            hpeso = heuristica(adyacente[0], destino)
            if adyacente[0] == destino:
                return analizando[4] + [(destino, analizando[1] + heuristica(analizando[0], destino))]
            elif adyacente[0] in cerrada:
                pass
            elif max(lbool):
                indice = lbool.index(True)
                if abierta[indice][1] < gpeso:
                    abierta[indice] = [(adyacente[0], gpeso, hpeso, gpeso+hpeso,
                                       analizando[4] + [(adyacente[0], gpeso)])]
            else:
                abierta += [(adyacente[0], gpeso, hpeso, gpeso+hpeso, analizando[4] + [(adyacente[0], gpeso)])]
    return cerrada